Imports

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import pyaudio
import speech_recognition as sr
import wave
import time
import threading


configure available audio device

In [ ]:
p = pyaudio.PyAudio()
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
RECORD_SECONDS = 2
WAVE_OUTPUT_FILENAME = "output.wav"
for i in range(p.get_device_count()):
    print (p.get_device_info_by_index(i))
    devices = p.get_device_info_by_index(i)
    if devices['name'] == "Stereo Mix (Realtek High Defini":
        device = p.get_device_info_by_index(i)
print(device,devices)

Open audio stream

In [3]:
stream = p.open(format = FORMAT,
                channels = CHANNELS,
                rate = RATE,
                input = True,
                input_device_index = device['index'],
                frames_per_buffer = CHUNK)

create webdriver object

In [4]:
# chrome driver path
driver_path = r'./chromedriver'
options = webdriver.ChromeOptions()
# User data directory path, it is required to attend gmeet using your id
options.add_argument("user-data-dir=C:\\Users\\Aakash\\AppData\\Local\\Google\\Chrome\\User Datatest")
driver = webdriver.Chrome(executable_path=driver_path, options=options)
driver.get("") #enter gmeet url here

Make mic/video disabled

In [5]:
# Make mic disabled
mic_enabled = driver.find_element_by_class_name('U26fgb.JRY2Pb.mUbCce.kpROve.uJNmj.HNeRed.QmxbVb')
mic_enabled.click()
# Make video disabled
vid_enabled = driver.find_element_by_class_name('U26fgb.JRY2Pb.mUbCce.kpROve.uJNmj.HNeRed.QmxbVb')
vid_enabled.click()

Click on "Ask to Join" button

In [ ]:
ask_to_join = driver.find_element_by_class_name('uArJ5e.UQuaGc.Y5sE8d.uyXBBb.xKiqt')
ask_to_join.click()
print(ask_to_join)

Focus on Message Box

In [7]:
msg = driver.find_element_by_class_name('uArJ5e.UQuaGc.kCyAyd.QU4Gid.foXzLb')
msg.click()

Focus on Chat Input

In [ ]:
chat = driver.find_element_by_class_name('ThdJC.kaAt2.c0XF8e.s7PhZd.sVoT0c.diMUPd.zCU1Sc')
chat.click()

Main utility to listen for people audio and respond

In [ ]:
aud_width = p.get_sample_size(FORMAT)
def func1(RATE,CHUNK, RECORD_SECONDS,WAVE_OUTPUT_FILENAME,CHANNELS,FORMAT):
    frames = []
    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)
    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()

def func2(WAVE_OUTPUT_FILENAME):
    r = sr.Recognizer()
    with sr.AudioFile(WAVE_OUTPUT_FILENAME) as source:
        # listen for the data (load audio to memory)
        audio_data = r.record(source)
        # recognize (convert from speech to text)
        text = r.recognize_google(audio_data, language = 'en-IN', show_all = True)
    try:
        print(text['alternative'])
        for i in text['alternative']:
            if i['transcript'].upper() == "AAKASH" or i['transcript'].upper() == "AKASH":
                print("present sir")
                msg_box = driver.find_element_by_class_name('KHxj8b.tL9Q4c')
                msg_box.click()
                msg_box.clear()
                msg_box.send_keys("Present Mam")
                send_msg = driver.find_element_by_class_name('uArJ5e.Y5FYJe.cjq2Db.IOMpW.Cs0vCd')
                send_msg.click()
    except:
        print("hehe")
        pass
while 1:
    t1 = threading.Thread(target=func1,args=(RATE,CHUNK, RECORD_SECONDS,WAVE_OUTPUT_FILENAME,CHANNELS,FORMAT,))
    t2 = threading.Thread(target=func2,args=(WAVE_OUTPUT_FILENAME,))
    t1.start()
    t2.start()
    t1.join()
    t2.join()
    print("threading loop infinite")
    